In [472]:
# Load the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [473]:
# Read the datasets
train = pd.read_csv('Train_Kaggle.csv').query('ProductCategory == "WomenClothing"') # 84 rows and 4 cols
test = pd.read_csv('Test_Kaggle.csv').query('ProductCategory == "WomenClothing"')
#train = pd.read_csv('Train_Kaggle.csv')
#test = pd.read_csv('Test_Kaggle.csv')
weather = pd.read_excel("WeatherData.xlsx")
economic = pd.read_excel("MacroEconomicData.xlsx")
holiday = pd.read_excel("Events_HolidaysData.xlsx")
economic.shape

(96, 18)

In [474]:
print(holiday.columns)

Index(['Year', 'MonthDate', 'Event', 'DayCategory'], dtype='object')


In [475]:
# Check missing values in each data set
train.isnull().sum() # There are 4 missing values in women sales
train
print(train.shape, test.shape)

(72, 4) (12, 4)


In [476]:
train = train.fillna(train.median())

In [477]:
# Pre process train dataset for ease
def preprocess(dataset):
    dataset.columns = ['year','month','productcategory','target'] #Renamed the columns
    dataset[['year','month']] = dataset[['year','month']].astype(str)
    dataset['year-month'] = dataset.year.str[:4] + "-" + dataset.month.str[:2] # Add a new column to join later
    dataset = dataset[['target','year-month']]
    return dataset
train = preprocess(train)
test = preprocess(test)
print(train.shape, test.shape)

(72, 2) (12, 2)


In [478]:
# Cleaned the weatherdata and downloaded onto local.
weather_2009 = pd.read_csv("WeatherData2009.csv")
weather_2010 = pd.read_csv("WeatherData2010.csv")
weather_2011 = pd.read_csv("WeatherData2011.csv")
weather_2012 = pd.read_csv("WeatherData2012.csv")
weather_2013 = pd.read_csv("WeatherData2013.csv")
weather_2014 = pd.read_csv("WeatherData2014.csv")
weather_2015 = pd.read_csv("WeatherData2015.csv")
weather_2016 = pd.read_csv("WeatherData2016.csv")

In [479]:
# Process weather
def process_weather(df):
    # Rename columns for ease
    df.columns = ['year', 'month','day','temp_high','temp_avg','temp_low','dew_point_high','dew_point_avg',
                  'dew_point_low','humidity_high','humidity_avg','humidity_low','pressure_high','pressure_avg','pressure_low',
                 'visibility_high','visibility_avg','visibility_low','wind_low','wind_avg','wind_high','precip_sum','weatherevent']
    # Convert year and day into strings
    df[['year', 'day','month','precip_sum','weatherevent']] = df[['year', 'day','month','precip_sum','weatherevent']].astype(str)
    
    # Convert all other columns to numerics
    df[['temp_high','temp_avg','temp_low','dew_point_high','dew_point_avg',
                  'dew_point_low','humidity_high','humidity_avg','humidity_low','pressure_high','pressure_avg','pressure_low',
                 'visibility_high','visibility_avg','visibility_low','wind_low','wind_avg','wind_high']] = df[['temp_high','temp_avg','temp_low','dew_point_high','dew_point_avg',
                  'dew_point_low','humidity_high','humidity_avg','humidity_low','pressure_high','pressure_avg','pressure_low',
                 'visibility_high','visibility_avg','visibility_low','wind_low','wind_avg','wind_high']].apply(pd.to_numeric, errors = "coerce")
    
    # Fill missing values with median of that column # Try mode/mean
    df = df.fillna(df.median())

    # Scaled all the numerical attributes using range method (simple custom function below)
    df_num = df.select_dtypes(include=[np.number]) # Only include numeric columns
    df_norm = (df_num - df_num.mean()) / (df_num.max() - df_num.min()) # Scale the numeric col
    df[df_norm.columns] = df_norm # Add them back to the original dataframe
    return df

In [480]:
def aggregate_weather(df):
    #Convert year to int
    df['year'] = df['year'].astype(int)
    # Group by month all the weather attributes
    df_aggregate_weather = df.groupby('month').mean() 
    
    #Insert the month columns
    month = ["4", "8", "12", "2", "1","7", "6", "3", "5", "11", "10", "9"] 
    df_aggregate_weather.insert(loc=0, column='month', value=month)
    
    # Convert year back for str
    df_aggregate_weather['year'] = df_aggregate_weather['year'].astype(str)
    
    # Insert a new column 'year-month' that would be useful for future join
    df_aggregate_weather['year-month'] = df_aggregate_weather.year.str[:4] + "-" + df_aggregate_weather.month.str[:2]
    
    # Convert month back to int
    df_aggregate_weather['month'] = df_aggregate_weather['month'].astype(int)
    return df_aggregate_weather

In [481]:
df1 = aggregate_weather(process_weather(weather_2009)) # 12 rows 21 columns
df2 = aggregate_weather(process_weather(weather_2010)) # 12 rows 21 columns
df3 = aggregate_weather(process_weather(weather_2011)) # 12 rows 21 columns
df4 = aggregate_weather(process_weather(weather_2012)) # 12 rows 21 columns
df5 = aggregate_weather(process_weather(weather_2013)) # 12 rows 21 columns
df6 = aggregate_weather(process_weather(weather_2014)) # 12 rows 21 columns
df7 = aggregate_weather(process_weather(weather_2015)) # 12 rows 21 columns
df8 = aggregate_weather(process_weather(weather_2016)) # 12 ros 21 columns
# clean_weather(weather_2016)

In [482]:
# Merge with the train set
df_new = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8])
train_weather = train.merge(df_new, on='year-month', how = 'left')
test_weather = test.merge(df_new, on='year-month', how = 'left')

print(train_weather.shape, test_weather.shape)

(72, 22) (12, 22)


In [483]:
#train_weather #184 ros 22 cols
#test_weather #12 rows 22 cols

In [484]:
X_train_weather = train_weather.drop(['year-month','year'], axis =1).iloc[:,1:]
Y = train_weather[['target']] #Y = train["target"]
print(X_train_weather.shape, Y.shape)

(72, 19) (72, 1)


In [485]:
X_test_weather = test_weather.drop(['year-month','year','target'], axis =1)
X_test_weather.shape

(12, 19)

In [486]:
# Ecocnomic Data

In [487]:
def process_economic(df):
    # Rename columns for ease
    df.columns = ['year-month', 'gdp','realgdp','cpi','party',
                  'unemployment','interestrate',
                  'financerate','wagesperhour','adv',
                  'cotton','cottonchange','planted','harvested','yield',
                 'production','usage','exports']
    # Convert year into strings
    df[['year-month']] = df[['year-month']].astype(str)
    
    # Drop Adversitising, Party
    df = df.drop(['adv','party'], axis=1)
    # Convert all other columns to numerics
    df[['gdp','realgdp','cpi','unemployment','interestrate',
       'financerate','wagesperhour','cotton','cottonchange','planted','harvested','yield',
                 'production','usage','exports']] = df[['gdp','realgdp','cpi','unemployment','interestrate',
       'financerate','wagesperhour','cotton','cottonchange','planted','harvested','yield',
                 'production','usage','exports']].apply(pd.to_numeric, errors = "coerce")
    
    # Fill missing values with median of that column # Try mode/mean
    df = df.fillna(df.median())

    # Scaled all the numerical attributes using range method (simple custom function below)
    df_num = df.select_dtypes(include=[np.number]) # Only include numeric columns
    df_norm = (df_num - df_num.mean()) / (df_num.max() - df_num.min()) # Scale the numeric col
    df[df_norm.columns] = df_norm # Add them back to the original dataframe
    return df

In [488]:
X_train_weather.shape

(72, 19)

In [489]:
#train_weath_econ.shape

In [490]:
# Merging Economic Data with train_weather

#train_weather_economic = pd.concat([X_train_weather, X_test_weather], axis=0)
train_weath_econ = pd.concat([train_weather, process_economic(economic.iloc[0:36,:])], axis=1)
train_weath_econ = train_weath_econ.drop(['year-month','year'],axis=1) # 84 rows & 36 cols
# # Merging Economic Data with test_weather
#test_weath_econ = pd.read_excel("MacroEconomicData.xlsx") # 12 rows 36 cols
test_weath_econ = pd.concat([test_weather, process_economic(economic.iloc[0:36,:])], axis=1)
#test_weath_econ = test_weath_econ.drop(['Year-Month', 'PartyInPower', 'AdvertisingExpenses (in Thousand Dollars)'], axis=1) #12 rows & 35 cols

print(train_weath_econ.shape, test_weath_econ.shape)

(72, 35) (36, 38)


/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [491]:
# Create X and Y for weather_economic merge
X_train_eco = train_weath_econ.iloc[:,1:] # 84 rows 35 cols
Y_eco = train_weath_econ[['target']] # 84 rows 1 cols

print(X_train_eco.shape, Y_eco.shape)

(72, 34) (72, 1)


In [492]:
# Modelling
#Split the data

x_train = X_train_eco.iloc[0:36,:]
x_val = X_train_eco.iloc[36:,:]
y_train = Y_eco.iloc[0:36,:]
y_val = Y_eco.iloc[36:,:]
#y_val

#dtrain = xgb.DMatrix(x_train, label=y_train)
#dvalid = xgb.DMatrix(x_val, label=y_val)
# dtest = xgb.DMatrix(test[feature_names].values)
#watchlist = [(dtrain, 'train'), (dvalid, 'valid')]


In [493]:
print(X_train_eco.shape, Y_eco.shape)

(72, 34) (72, 1)


In [494]:
test_weath_econ.shape

(36, 38)

In [495]:
test_weath_econ.shape

(36, 38)

In [496]:
def process_eco(df):
    #Convert year to int
    df = df.drop(['year-month'], axis=1)
    #df = df.drop(['target'], axis=1)
    df = df.drop(['year'], axis=1)
    #df = df.drop(['PartyInPower'], axis=1)
    #df['AdvertisingExpenses (in Thousand Dollars)'] = df['AdvertisingExpenses (in Thousand Dollars)'].replace('?', np.nan)
    #df['AdvertisingExpenses (in Thousand Dollars)'] = df['AdvertisingExpenses (in Thousand Dollars)'].fillna(df['AdvertisingExpenses (in Thousand Dollars)'].median())
    #df['AdvertisingExpenses (in Thousand Dollars)'] = df['AdvertisingExpenses (in Thousand Dollars)'].astype(int)    
    #df['PartyInPower'] = df['PartyInPower'].astype(int)    
    return df

In [497]:
test_weath_econ = process_eco(test_weath_econ)
#test_weath_econ.dtypes

In [498]:
#test_weath_econ.head()

In [499]:
#X_train_eco.head()

In [500]:
# Holiday Data

In [501]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
def process_holiday(df):
    # Rename columns for ease
    df.columns = ['year', 'month-date','event','dat-category']
    # Convert year into strings
    #df[['year-month']] = df[['year-month']].astype(str)
    
    # Drop Adversitising, Party
    #df = df.drop(['adv','party'], axis=1)
    # Convert all other columns to numerics
    df[['year', 'month-date','event','dat-category']].apply(pd.to_numeric, errors = "coerce")
    
    # Fill missing values with median of that column # Try mode/mean
    df = df.fillna(df.median())
      
    le.fit(df['event'])
    df['event'] = le.transform(df['event'])
    le.fit(df['dat-category'])
    df['dat-category'] = le.transform(df['dat-category'])
    
    # Scaled all the numerical attributes using range method (simple custom function below)
    df_num = df.select_dtypes(include=[np.number]) # Only include numeric columns
    df_norm = (df_num - df_num.mean()) / (df_num.max() - df_num.min()) # Scale the numeric col
    df[df_norm.columns] = df_norm # Add them back to the original dataframe
    return df

In [502]:
#print(process_holiday(holiday.iloc[0:36,:]).columns)

In [503]:
# Merging Hoilday Data with train_eco

train_weath_holiday = pd.concat([train_weath_econ, process_holiday(holiday.iloc[0:36,:])], axis=1)
train_weath_holiday = train_weath_holiday.drop(['month-date','year'],axis=1) # 84 rows & 36 cols
# # Merging Hoilday Data with test_eco
test_weath_holiday = pd.concat([test_weath_econ, process_holiday(holiday.iloc[0:36,:])], axis=1)

print(train_weath_holiday.shape, test_weath_holiday.shape)

(72, 37) (36, 39)


In [504]:
def process_holi(df):
    #Convert year to int
    df = df.drop(['month-date'], axis=1)
    df = df.drop(['target'], axis=1)
    df = df.drop(['year'], axis=1)
    
    le.fit(df['event'])
    df['event'] = le.transform(df['event'])
    le.fit(df['dat-category'])
    df['dat-category'] = le.transform(df['dat-category'])
    
    df = df.fillna(df.median())
    
    return df

In [505]:
Y_holi_test = test_weath_holiday[['target']]

In [506]:
test_weath_holiday = process_holi(test_weath_holiday)
#test_weath_holiday.head

In [507]:
#test_weath_holiday.dtypes

In [508]:
# Create X and Y for weather_economic_holiday merge
X_train_holi = train_weath_holiday.iloc[:,1:] 
X_train_holi = X_train_holi.fillna(X_train_holi.median())
Y_holi = train_weath_holiday[['target']]

print(X_train_holi.shape, Y_eco.shape)

(72, 36) (72, 1)


In [509]:
# rmse function
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [543]:
#test_weath_holiday.dtypes
#test_weath_holiday = test_weath_holiday.drop(['month'],axis=1)
#X_train_holi = X_train_holi.drop(['month'],axis=1)

print(X_train_holi.cov())

                     month  temp_high  temp_avg  temp_low  dew_point_high  \
month            12.084507   0.206042  0.228645  0.246387        0.250651   
temp_high         0.206042   0.045443  0.046003  0.045079        0.042897   
temp_avg          0.228645   0.046003  0.046739  0.045974        0.043815   
temp_low          0.246387   0.045079  0.045974  0.045425        0.043393   
dew_point_high    0.250651   0.042897  0.043815  0.043393        0.042036   
dew_point_avg     0.258384   0.043420  0.044406  0.044049        0.042587   
dew_point_low     0.272150   0.044712  0.045786  0.045495        0.044003   
humidity_high     0.102285   0.013945  0.014542  0.014801        0.015206   
humidity_avg      0.109455   0.008931  0.009585  0.010078        0.010720   
humidity_low      0.110666   0.004522  0.005188  0.005854        0.006694   
pressure_high     0.021212  -0.008490 -0.008565 -0.008406       -0.008056   
pressure_avg      0.041427  -0.002664 -0.002654 -0.002580       -0.002539   

In [542]:
# Simple Xgboost
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.model_selection import GridSearchCV

# XGBRegressor
# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}

model_xgbr = XGBRegressor(nthread=-1) 
model_xgbr = GridSearchCV(model_xgbr, params)
model_xgbr.fit(X_train_holi, Y_holi)

y_test_xgb = model_xgbr.best_estimator_.predict(test_weath_holiday)
y_test_xgb_score = model_xgbr.score(test_weath_holiday, y_test_xgb)
#print(test_weath_holiday.values.shape)
rmse_xgb_value = rmse(y_test_xgb,test_weath_holiday.values)

# RandomForestRegressor
model_rfr = RandomForestRegressor(100, oob_score = 1,n_jobs = 1,random_state =42)
model_rfr.fit(X_train_holi, Y_holi)

y_test_rfr = model_rfr.predict(test_weath_holiday)
y_test_rfr_score = model_rfr.score(test_weath_holiday, y_test_xgb)
rmse_rfr_value = rmse(y_test_rfr,test_weath_holiday.values)

# CatBoostRegressor
model_cbr = CatBoostRegressor(iterations=2, learning_rate=1, depth=2)
model_cbr.fit(X_train_holi, Y_holi)

y_test_cbr = model_cbr.predict(test_weath_holiday)
y_test_cbr_score = model_cbr.score(test_weath_holiday, y_test_cbr)
rmse_cbr_value = rmse(y_test_cbr, test_weath_holiday.values)

# LogisticRegression
model_lr = LogisticRegression()
model_lr.fit(X_train_holi, Y_holi)

y_test_lr = model_lr.predict(test_weath_holiday)
y_test_lr_score = model_lr.score(test_weath_holiday, y_test_lr)
rmse_lr_value = rmse(y_test_lr, test_weath_holiday.values)

# DecisionTreeRegressor
model_dtr = DecisionTreeRegressor(random_state=0)
model_dtr.fit(X_train_holi, Y_holi)

y_test_dtr = model_dtr.predict(test_weath_holiday)
y_test_dtr_score = model_dtr.score(test_weath_holiday, y_test_dtr)
rmse_dtr_value = rmse(y_test_dtr, test_weath_holiday.values)

# LinearRegression
model_lrm = LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
model_lrm.fit(X_train_holi, Y_holi)

y_test_lrm = model_lrm.predict(test_weath_holiday)
y_test_lrm_score = model_lrm.score(test_weath_holiday, y_test_lrm)
rmse_lrm_value = rmse(y_test_lrm, test_weath_holiday.values)

# GradientBoostingRegressor
model_gbr = GradientBoostingRegressor()
model_gbr.fit(X_train_holi, Y_holi)

y_test_gbr = model_gbr.predict(test_weath_holiday)
y_test_gbr_score = model_gbr.score(test_weath_holiday, y_test_gbr)
rmse_gbr_value = rmse(y_test_gbr, test_weath_holiday.values)

# SVR
model_svr = SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='auto',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
model_svr.fit(X_train_holi, Y_holi)

y_test_svr = model_svr.predict(test_weath_holiday)
y_test_svr_score = model_svr.score(test_weath_holiday, y_test_svr)
rmse_svr_value = rmse(y_test_svr, test_weath_holiday.values)

# AdaBoostRegressor
model_adb = AdaBoostRegressor()
model_adb.fit(X_train_holi, Y_holi)

y_test_adb = model_adb.predict(test_weath_holiday)
y_test_adb_score = model_adb.score(test_weath_holiday, y_test_adb)
rmse_adb_value = rmse(y_test_adb, test_weath_holiday.values)

# #ARIMA
# #ts = pd.Series(np.random.randn(500), index=pd.date_range('2010-01-01', periods=500))
# history = [x for x in X_train_holi]
# #print(history)
# model_arma = ARIMA(np.asarray(history, dtype=float).mean(), order=(5,1,0))
# model_arma.fit(X_train_holi, Y_holi)

# y_test_arma = model_arma.predict(test_weath_holiday)
# y_test_arma_score = model_arma.score(test_weath_holiday, y_test_arma)
# rmse_arma_value = rmse(y_test_arma, test_weath_holiday.values)


# Model mapping 
models = pd.DataFrame({
    'Model': ['XGBRegressor', 'RandomForestRegressor', 'CatBoostRegressor',
              'LogisticRegression', 'DecisionTreeRegressor', 'LinearRegression',
              'GradientBoostingRegressor', 'SVR', 'AdaBoostRegressor'],
#     'Score': [y_test_xgb_score, y_test_rfr_score, y_test_cbr_score,
#               y_test_lr_score, y_test_dtr_score, y_test_lrm_score,
#               y_test_gbr_score, y_test_svr_score, y_test_adb_score],
    'RMSE': [rmse_xgb_value, rmse_rfr_value, rmse_cbr_value,
             rmse_lr_value, rmse_dtr_value, rmse_lrm_value,
             rmse_gbr_value, rmse_svr_value, rmse_adb_value]})
models.index = np.arange(1, len(models)+1) # starting index from 1
models.sort_values(by='RMSE', ascending=True)

# results_xgb = pd.DataFrame(data={'Sales(In ThousandDollars)':y_test_xgb}) 
# results_xgb.index = np.arange(1, len(results_xgb)+1) # starting index from 1
# results_xgb.index.name = "Year" # index header name
# results_xgb.to_csv('test_results.csv')
# results_xgb


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=Tru

0:	learn: 801.6674108	total: 1.53ms	remaining: 1.53ms
1:	learn: 582.2141568	total: 2.61ms	remaining: 0us


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Model,RMSE
6,LinearRegression,1562.202754
5,DecisionTreeRegressor,2920.784460
1,XGBRegressor,2984.297660
4,LogisticRegression,2988.393152
7,GradientBoostingRegressor,3019.282850
2,RandomForestRegressor,3028.455601
3,CatBoostRegressor,3222.148990
8,SVR,3242.134289
9,AdaBoostRegressor,3252.061496
